<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q keras

In [18]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = '/content/fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:60000]



In [5]:
print(pairs[11117]) jkcashiefhiwei

['A recurrent neural network is a class of artificial neural networks ', 'Un réseau de neurones récurrent est une classe de réseaux de neurones artificiels']


In [19]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = "?" + "¡" + '¿'+"."+"!"+"." ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [20]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [21]:

# english_sentences = [clean_sentence(pair[0]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]
# french_sentences = [clean_sentence(pair[1]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]

# # Tokenize words
# fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
# eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

# print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
# print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# # Check language length
# french_vocab = len(fra_text_tokenizer.word_index) + 1
# english_vocab = len(eng_text_tokenizer.word_index) + 1
# print("french vocabulary is of {} unique words".format(french_vocab))
# print("English vocabulary is of {} unique words".format(english_vocab))


# Clean sentences
# english_sentences = [clean_sentence(pair[0]) for pair in pairs]
# french_sentences = [clean_sentence(pair[1]) for pair in pairs]
english_sentences = [clean_sentence(pair[0]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]
french_sentences = [clean_sentence(pair[1]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]


# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length spanish sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length spanish sentence: 14
Maximum length english sentence: 12
Spanish vocabulary is of 13638 unique words
English vocabulary is of 7480 unique words


In [7]:
# max_french_len = int(len(max(fra_text_tokenized,key=len)))
# max_english_len = int(len(max(eng_text_tokenized,key=len)))

# fra_pad_sentence = pad_sequences(fra_text_tokenized,max_french_len, padding = "post")
# eng_pad_sentence = pad_sequences(eng_text_tokenized,max_english_len, padding = "post")




# fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
# eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [22]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [23]:
# input_sequence = Input(shape=(max_french_len,))
# embedding=Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)

input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)

In [24]:
# encoder = LSTM(64, return_sequences=False)(embedding)
# r_vec = RepeatVector(15)(encoder)
# decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
# logits = TimeDistributed(Dense(english_vocab))(decoder)

input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [25]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 14)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 14, 256)           3491328   
                                                                 
 lstm_2 (LSTM)               (None, 64)                82176     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 12, 64)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 12, 64)            33024     
                                                                 
 time_distributed_1 (TimeDis  (None, 12, 7480)         486200    
 tributed)                                                 

In [ ]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(fra_pad_sentence) / BATCH_SIZE
SAVE_PERIOD = 10


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model/weights1.{epoch:02d}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))

In [26]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=32, epochs=70)


Epoch 1/70
1875/1875 [==============================] - 25s 9ms/step - loss: 2.1979 - accuracy: 0.7037
Epoch 2/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.9422 - accuracy: 0.7084
Epoch 3/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.9163 - accuracy: 0.7091
Epoch 4/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.7336 - accuracy: 0.7299
Epoch 5/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.5062 - accuracy: 0.7564
Epoch 6/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.3447 - accuracy: 0.7755
Epoch 7/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.2248 - accuracy: 0.7891
Epoch 8/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.1312 - accuracy: 0.7992
Epoch 9/70
1875/1875 [==============================] - 17s 9ms/step - loss: 1.0523 - accuracy: 0.8076
Epoch 10/70
1875/1875 [==============================] - 17s 9ms/step - l

In [27]:
enc_dec_model.save('finalfinal.h5')

In [ ]:
import tensorflow as tf
new_enc_dec_model = tf.keras.models.load_model('my_model_v2.h5')



In [31]:
x=["Un réseau de neurones récurrent est une classe de réseaux de neurones artificiels"]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, max_french_len, padding = "post")

# print(y)
z=np.reshape(z,(1,max_french_len,1))

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x[0]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(z)[0], eng_text_tokenizer))

# print(fra_pad_sentence[50001:50002].shape)
# print(z.shape)




The french sentence is: Un réseau de neurones récurrent est une classe de réseaux de neurones artificiels
The predicted sentence is :
a recurrent neural network is a class of artificial neural networks 


In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs

In [ ]:
tfjs.converters.save_keras_model(new_enc_dec_model, 'content/model')

In [ ]:
!zip -r model.zip content/model

  adding: content/model/ (stored 0%)
  adding: content/model/model.json (deflated 77%)
  adding: content/model/group1-shard2of3.bin (deflated 7%)
  adding: content/model/group1-shard1of3.bin (deflated 7%)
  adding: content/model/group1-shard3of3.bin (deflated 7%)
